# ZenML Practice

## Setup ZenML Dashboard

### Define a stack
Following tutorial from https://mlstacks.zenml.io/getting-started/quickstart/k3d#defining-our-stack

In [1]:
# Writing to file
with open("quickstart_stack.yaml", "w") as file1:
    # Writing data to a file
    file1.write("""spec_version: 1
spec_type: stack
name: "quickstart_stack"
provider: k3d
default_region: "local"
default_tags:
  deployed-by: "mlstacks"
components:
  - simple_component_minio.yaml""")

In [2]:
# writing a component
with open("simple_component_minio.yaml", "w") as file1:
    # Writing data to a file
    file1.write("""spec_version: 1
spec_type: component
component_type: "artifact_store"
component_flavor: "minio"
name: "quickstart_minio_bucket"
provider: k3d
metadata:
  config:
    bucket_name: "quickstart_minio_bucket"
  tags:
    deployed-by: "mlstacks"
  region: "local"
""")

In [3]:
# deploy the stack
# CURRENTLY BROKEN because of a bug in mlstacks about k3d enum
!mlstacks deploy -f quickstart_stack.yaml

#MLstacks normally requires YAML files, but if used with zenml they are not needed

Deploying stack from 'quickstart_stack.yaml'...
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /home/vscode/.cache/pypoetry/virtualenvs/zenml-learning-bkVTvPcB-py3.10/bin/ │
│ mlstacks:8 in <module>                                                       │
│                                                                              │
│   5 from mlstacks.cli.cli import cli                                         │
│   6 if __name__ == "__main__":                                               │
│   7 │   sys.argv[0] = re.sub(r"(-script\.pyw|\.exe)?$", "", sys.argv[0])     │
│ ❱ 8 │   sys.exit(cli())                                                      │
│   9                                                                          │
│                                                                              │
│ ╭───────────────────────────────── locals ─────────────────────────────────╮ │
│ │ __annotations__ = {}                                     

In [13]:
# so instead lets do it with zenml
# TODO also broken for the same reason
#!zenml down
#!zenml up
#!zenml stack deploy -p k3d -n test

Usage: zenml stack deploy [OPTIONS]
Try 'zenml stack deploy --help' for help.

Error: Missing option '--region' / '-r'.


In [5]:
# run basic stack
!python basic_pipeline.py

Initiating a new run for the pipeline: my_pipeline.
Reusing registered version: (version: 1).
Executing a new run.
Using user: default
Using stack: default2
  orchestrator: default
  artifact_store: custom_local
Using cached version of step_1.
Linking artifact output to model None version None implicitly.
Step step_1 has started.
Caching disabled explicitly for step_2.
Step step_2 has started.
hello world
Step step_2 has finished in 0.058s.
Run my_pipeline-2023_11_09-13_13_58_775740 has finished in 0.296s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


In [8]:
# register a new docker orchestrator (does not work with local artifact store https://docs.zenml.io/v/docs/stacks-and-components/component-guide/artifact-stores/local)
!zenml orchestrator register local_docker --flavor=local_docker
# make a new stack using docker as the orchestrator instead (stacks cannot be edited so we need to make a new, and must include an artifact store)
!zenml stack update local_docker_orc -o local_docker -a custom_local # also sets it as current
!zenml stack set local_docker_orc
!python basic_pipeline.py

⠋ Registering orchestrator 'local_docker'...
⠙ Registering orchestrator 'local_docker'...
⠹ Registering orchestrator 'local_docker'...
⠸ Registering orchestrator 'local_docker'...
⠼ Registering orchestrator 'local_docker'...
⠴ Registering orchestrator 'local_docker'...
⠦ Registering orchestrator 'local_docker'...
⠦ Registering orchestrator 'local_docker'...

╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /home/vscode/.cache/pypoetry/virtualenvs/zenml-learning-bkVTvPcB-py3.10/bin/ │
│ zenml:8 in <module>                                                          │
│                                                                              │
│   5 from zenml.cli.cli import cli                                            │
│   6 if __name__ == "__main__":                                               │
│   7 │   sys.argv[0] = re.sub(r"(-script\.pyw|\.exe)?$", "", sys.argv[0])     │
│ ❱ 8 │   sys.exit(cli())                                               

In [4]:
# register the local image builder (avoid the future error)
!zenml image-builder register local_image_builder --flavor=local
!zenml stack register local_docker_local_image_builder -o local_docker -i local_image_builder --set -a default 
!python basic_pipeline.py

⠋ Registering image builder 'local_image_builder'...
⠙ Registering image builder 'local_image_builder'...
⠹ Registering image builder 'local_image_builder'...
⠸ Registering image builder 'local_image_builder'...
⠼ Registering image builder 'local_image_builder'...
⠴ Registering image builder 'local_image_builder'...
⠦ Registering image builder 'local_image_builder'...
⠧ Registering image builder 'local_image_builder'...
Successfully registered image_builder `local_image_builder`.
⠧ Registering image builder 'local_image_builder'...
You can display various ZenML entities including pipelines, runs, stacks and 
much more on the ZenML Dashboard. You can try it locally, by running `zenml up`,
or remotely, by deploying ZenML on the infrastructure of your choice.
⠧ Registering image builder 'local_image_builder'...
⠧ Registering image builder 'local_image_builder'...

⠋ Registering stack 'local_docker_local_image_builder'...
⠙ Registering stack 'local_docker_local_image_builder'...
⠹ Register

In [9]:
# this time use kubernates as the orchestrator
!yes | zenml integration install kubernetes
# this will deploy it in k3d, requires mlstacks if using latest version of zenml, needs a container registry
#https://www.allisonthackston.com/articles/local-docker-registry.html
#!docker run -d -p 5000:5000 --restart always --name registry registry:2 
!zenml container-registry register k3d_container_registry_5001http --flavor=default --uri="http://$(k3d registry list | grep -o 'k3d-.*localhost'):5001"
!zenml orchestrator deploy k8s_orchestrator_hostname --flavor=kubernetes --provider=k3d --region=local
#!zenml image-builder register local_image_builder --flavor=local
!zenml stack register orc_k8s -o k8s_orchestrator -i local_image_builder -c hostname_container_registry2 --set -a default 
!python basic_pipeline.py

Are you sure you want to install the following packages to the current 
environment?
⠦ Installing integrations...: ⠋ Installing integrations...
yes: standard output: Broken pipe
de6da478007a299fe2649e5e06f9edb162f15e9327dc6b1b2cc1b6b3729abdb8
⠋ Registering container registry 'hostname_container_registry'...
⠙ Registering container registry 'hostname_container_registry'...
⠹ Registering container registry 'hostname_container_registry'...
⠸ Registering container registry 'hostname_container_registry'...
⠼ Registering container registry 'hostname_container_registry'...
⠴ Registering container registry 'hostname_container_registry'...
⠦ Registering container registry 'hostname_container_registry'...
Successfully registered container_registry `hostname_container_registry`.
⠦ Registering container registry 'hostname_container_registry'...
You can display various ZenML entities including pipelines, runs, stacks and 
much more on the ZenML Dashboard. You can try it locally, by running `zenml u